In [3]:
import numpy as np
import pandas as pd

csv_file_path = '../data/LKA/WHO_SLK_adm_lvl1_dengue_1996-2004.csv'

In [4]:
df = pd.read_csv(csv_file_path, header=None, sep=",")
df.head()

,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,108,109,110,111
0,NaN,NaN,NaN,NaN,1996 month: 1,1996 month: 2,1996 month: 3,1996 month: 4,1996 month: 5,1996 month: 6,...,2004 month: 3,2004 month: 4,2004 month: 5,2004 month: 6,2004 month: 7,2004 month: 8,2004 month: 9,2004 month: 10,2004 month: 11,2004 month: 12
1,LKA,Sri Lanka,LK-1,Central Province,1,0,1,2,3,5,...,30,65,155,143,73,27,11,11,1,2
2,LKA,Sri Lanka,LK-5,Eastern Province,9,1,4,34,10,2,...,14,19,10,23,12,1,1,17,4,4
3,LKA,Sri Lanka,LK-7,North Central Province,0,0,1,3,0,0,...,3,15,125,97,52,21,16,8,0,3
4,LKA,Sri Lanka,LK-6,North Western Province,1,1,0,2,6,20,...,12,65,140,65,23,4,1,0,1,2


In [5]:
def convert_date(raw_date):
    import re
    result = None
    date_regex = r'(\d{4})\s+month:\s+(\d{1,2})'
    matches = re.search(date_regex, raw_date)
    
    if matches:
        year = matches.group(1)
        month = matches.group(2)
        if len(month) == 1:
            month = '0' + month
        result = '{0}{1}01'.format(year, month)
    
    return result

In [6]:
arr = df.as_matrix().T

dates_raw = arr[4:, 0]
dates = [convert_date(x) for x in dates_raw]

places_raw = arr[:4, :]

new_arr = np.array([])

for i in range(1, arr.shape[1]):
    place_data = arr[:4, i]
       
    dengue_data = arr[4:, i]
    for j in range(dengue_data.shape[0]):
        date = dates[j]
        dengue_cases = dengue_data[j]
        if dengue_cases == 'n/a':
            dengue_cases = -1
        
        new_row = np.array(place_data)
        new_row = np.append(new_row, date)
        new_row = np.append(new_row, dengue_cases)
        
        if new_arr.shape[0] == 0:
            new_arr = new_row
        else:
            new_arr = np.vstack((new_arr, new_row))

        
column_names = ['country_iso2_id', 'country_name', 'admin_lvl1_id', 'admin_lvl1_name', 'period_date', 'dengue_total_cases']
new_df = pd.DataFrame(new_arr, columns=column_names)
new_df = new_df.apply(lambda x: pd.to_numeric(x, errors='ignore'))

new_df.head()

,country_iso2_id,country_name,admin_lvl1_id,admin_lvl1_name,period_date,dengue_total_cases
0,LKA,Sri Lanka,LK-1,Central Province,19960101,1
1,LKA,Sri Lanka,LK-1,Central Province,19960201,0
2,LKA,Sri Lanka,LK-1,Central Province,19960301,1
3,LKA,Sri Lanka,LK-1,Central Province,19960401,2
4,LKA,Sri Lanka,LK-1,Central Province,19960501,3


In [7]:
new_df.to_csv('lka_denguenet_data_refactored.csv', index=False, header=True)